# Email Market Screener Report Generator (Cache-Based)

## 📊 Overview
This notebook generates automated market screener reports using cached CLOB data and distributes them via email. It integrates with the QuantsLab notification system to send professional HTML-formatted volume reports, embedded charts, and detailed data files directly to email recipients for comprehensive market analysis.

## 🎯 Objectives
1. **Data Loading**: Load cached candle data from CLOB data source
2. **Volume Analysis**: Calculate volume metrics and identify top performers
3. **Report Generation**: Create professional HTML-formatted volume reports with styling
4. **Email Integration**: Send reports via SMTP using notification system
5. **Chart Creation**: Generate embedded charts for email compatibility
6. **File Distribution**: Send CSV reports and chart files as email attachments

## 📋 Prerequisites
- Cached CLOB candle data (run data collection tasks first)
- Email SMTP configuration (server, credentials in .env)
- QuantsLab notification system setup
- Environment variables configured (.env file)
- Plotly and pandas for analysis

## ⚠️ Important Notes
- **Data Source**: Uses cached CLOB data instead of live database queries
- **Email Setup**: Requires SMTP server configuration and credentials
- **HTML Formatting**: Creates professional email-friendly HTML reports
- **File Attachments**: Charts and CSV files sent as email attachments
- **Rate Limiting**: Be mindful of email provider sending limits

## 📈 Expected Outputs
- Professional HTML-formatted volume reports sent via email
- Embedded volume charts (PNG format for email compatibility)
- Detailed CSV reports as email attachments
- Automated email distribution for team collaboration

In [1]:
# 🔌 Initialize QuantsLab Services
# Import and configure core services and notification system
import warnings
warnings.filterwarnings("ignore")

from core.data_sources.clob import CLOBDataSource
from core.notifiers import NotificationManager, NotificationMessage
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import os
from datetime import datetime, timezone
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

print("✅ QuantsLab modules imported successfully")
print("📊 Services available: CLOB data source, Notification system")
print("⚙️ Environment variables loaded from .env")

✅ QuantsLab modules imported successfully
📊 Services available: CLOB data source, Notification system
⚙️ Environment variables loaded from .env


# ⚙️ Configuration and Data Loading
# Set up data source and load cached market data

In [ ]:
# Configuration Parameters
CONNECTOR_NAME = "binance_perpetual"  # Exchange connector to use
QUOTE_ASSET = "USDT"                  # Quote currency for trading pairs
INTERVAL = "15m"                       # Candle interval for analysis
FETCH_FRESH_DATA = False              # Set to True to fetch fresh data from exchange
DAYS = 7                              # Days of historical data (when fetching fresh)
TOP_PAIRS_COUNT = 15                  # Number of top pairs to include in report

# 📧 Notifier Recipient Override Options (all optional - leave None to use .env defaults)
# ===================================================================================

# Email Recipients Override
CUSTOM_EMAIL_RECIPIENTS = None  # Example: ["analyst@company.com", "trader@company.com"]

# Telegram Chat IDs Override  
CUSTOM_TELEGRAM_CHAT_IDS = None  # Example: ["-1001234567890", "123456789"]

# Slack Channels Override
CUSTOM_SLACK_CHANNELS = None  # Example: ["#trading-alerts", "#risk-management"]
CUSTOM_SLACK_WEBHOOKS = None  # Example: ["https://hooks.slack.com/services/T00/B00/XXX"]

# Discord Webhooks Override
CUSTOM_DISCORD_WEBHOOKS = None  # Example: ["https://discord.com/api/webhooks/123/ABC"]

# Initialize CLOB data source
clob = CLOBDataSource()

# Initialize notification manager
notification_manager = NotificationManager()
enabled_notifiers = notification_manager.get_enabled_notifiers()

print(f"📊 Configuration loaded:")
print(f"  - Connector: {CONNECTOR_NAME}")
print(f"  - Quote Asset: {QUOTE_ASSET}")
print(f"  - Interval: {INTERVAL}")
print(f"  - Top Pairs: {TOP_PAIRS_COUNT}")

print(f"\n📧 Recipient Override Configuration:")
if CUSTOM_EMAIL_RECIPIENTS:
    print(f"  - Email: {', '.join(CUSTOM_EMAIL_RECIPIENTS)}")
else:
    print(f"  - Email: Using .env defaults")

if CUSTOM_TELEGRAM_CHAT_IDS:
    print(f"  - Telegram: {len(CUSTOM_TELEGRAM_CHAT_IDS)} custom chat IDs")
else:
    print(f"  - Telegram: Using .env default chat_id")

if CUSTOM_SLACK_CHANNELS or CUSTOM_SLACK_WEBHOOKS:
    channels = f"{len(CUSTOM_SLACK_CHANNELS)} channels" if CUSTOM_SLACK_CHANNELS else "default channel"
    webhooks = f"{len(CUSTOM_SLACK_WEBHOOKS)} webhooks" if CUSTOM_SLACK_WEBHOOKS else "default webhook"
    print(f"  - Slack: {channels}, {webhooks}")
else:
    print(f"  - Slack: Using .env defaults")

if CUSTOM_DISCORD_WEBHOOKS:
    print(f"  - Discord: {len(CUSTOM_DISCORD_WEBHOOKS)} custom webhooks")
else:
    print(f"  - Discord: Using .env default webhook")

print(f"\n📧 Enabled notifiers: {', '.join(enabled_notifiers) if enabled_notifiers else 'None configured'}")

In [3]:
# 📊 Load Market Data from Cache
# Load cached candle data or fetch fresh data if configured

if FETCH_FRESH_DATA:
    print("🔄 Fetching fresh data from exchange...")
    # Get trading pairs
    trading_rules = await clob.get_trading_rules(CONNECTOR_NAME)
    trading_pairs = [
        pair for pair in trading_rules.get_all_trading_pairs() 
        if pair.split("-")[1] == QUOTE_ASSET
    ]
    
    # Fetch candle data
    candles = await clob.get_candles_batch_last_days(
        CONNECTOR_NAME, trading_pairs, INTERVAL, DAYS, 2, 1.0
    )
    
    # Save to cache
    clob.dump_candles_cache()
    print(f"✅ Fetched data for {len(candles)} trading pairs")
else:
    print("📂 Loading data from cache...")
    clob.load_candles_cache()
    print("✅ Cache loaded successfully")

# Filter cached data for our requirements
candles = [
    candle for key, candle in clob.candles_cache.items()
    if key[0] == CONNECTOR_NAME and key[2] == INTERVAL and key[1].endswith(f"-{QUOTE_ASSET}")
]

print(f"📈 Found {len(candles)} datasets matching criteria:")
print(f"  - Connector: {CONNECTOR_NAME}")
print(f"  - Interval: {INTERVAL}")
print(f"  - Quote Asset: {QUOTE_ASSET}")

if len(candles) == 0:
    print("⚠️  No cached data found. Please:")
    print("   1. Set FETCH_FRESH_DATA = True to fetch new data")
    print("   2. Or run data collection tasks to populate cache")
else:
    sample_pairs = [candle.trading_pair for candle in candles[:5]]
    print(f"📊 Sample pairs: {', '.join(sample_pairs)}{'...' if len(candles) > 5 else ''}")

📂 Loading data from cache...
✅ Cache loaded successfully
📈 Found 21 datasets matching criteria:
  - Connector: binance_perpetual
  - Interval: 15m
  - Quote Asset: USDT
📊 Sample pairs: XRP-USDT, ETH-USDT, DOGE-USDT, DOT-USDT, OP-USDT...


# 📊 Market Analysis and Volume Metrics
# Calculate volume metrics and create market ranking

In [4]:
# 💹 Calculate Volume Metrics for Market Screening
# Process each trading pair and calculate comprehensive volume metrics

volume_report = []

for candle in candles:
    try:
        df = candle.data.copy()
        
        if df.empty or len(df) < 24:  # Need at least 24 hours of data
            continue
            
        # Calculate volume metrics
        latest_price = df['close'].iloc[-1]
        
        # Volume analysis (last 24 hours)
        df_24h = df.tail(24) if len(df) >= 24 else df
        volume_24h_base = df_24h['volume'].sum()
        volume_24h_usd = volume_24h_base * latest_price
        
        # Price change analysis
        price_24h_ago = df_24h['close'].iloc[0] if len(df_24h) > 1 else latest_price
        price_change_24h = ((latest_price - price_24h_ago) / price_24h_ago * 100) if price_24h_ago > 0 else 0
        
        # Volatility analysis
        price_std = df_24h['close'].std()
        volatility_pct = (price_std / latest_price * 100) if latest_price > 0 else 0
        
        # High/Low analysis
        high_24h = df_24h['high'].max()
        low_24h = df_24h['low'].min()
        
        # Current position in 24h range (0 = at low, 1 = at high)
        range_24h = high_24h - low_24h
        position_in_range = ((latest_price - low_24h) / range_24h) if range_24h > 0 else 0.5
        
        # Volume trend (compare recent vs earlier volume)
        recent_volume = df.tail(6)['volume'].mean()  # Last 6 hours average
        earlier_volume = df.iloc[-24:-6]['volume'].mean() if len(df) >= 24 else recent_volume
        volume_trend = ((recent_volume - earlier_volume) / earlier_volume * 100) if earlier_volume > 0 else 0
        
        # Calculate a composite score (volume * volatility * price momentum)
        momentum_factor = abs(price_change_24h) / 100  # Normalize price change
        volume_factor = volume_24h_usd / 1000000  # Normalize to millions USD
        volatility_factor = volatility_pct / 100  # Normalize volatility
        
        composite_score = volume_factor * volatility_factor * (1 + momentum_factor)
        
        volume_report.append({
            'trading_pair': candle.trading_pair,
            'connector_name': CONNECTOR_NAME,
            'price': latest_price,
            'volume_24h_base': volume_24h_base,
            'volume_24h_usd': volume_24h_usd,
            'price_change_24h_pct': price_change_24h,
            'volatility_pct': volatility_pct,
            'high_24h': high_24h,
            'low_24h': low_24h,
            'position_in_range': position_in_range,
            'volume_trend_pct': volume_trend,
            'composite_score': composite_score,
            'data_points': len(df)
        })
        
    except Exception as e:
        print(f"⚠️  Error processing {candle.trading_pair}: {e}")
        continue

# Convert to DataFrame and sort by volume
df = pd.DataFrame(volume_report)

if df.empty:
    print("❌ No data could be processed for volume analysis")
    print("💡 Please check if cached data is available or fetch fresh data")
else:
    # Sort by volume (USD) and add ranking
    df = df.sort_values('volume_24h_usd', ascending=False).reset_index(drop=True)
    df['volume_rank'] = range(1, len(df) + 1)
    
    print(f"📊 Volume Analysis Complete:")
    print(f"  - Total pairs analyzed: {len(df)}")
    print(f"  - Top volume: ${df['volume_24h_usd'].max():,.0f}")
    print(f"  - Date range: {INTERVAL} candles")
    
    # Show top performers
    print(f"\n🏆 TOP {min(10, len(df))} BY VOLUME:")
    print("-" * 70)
    
    display_cols = ['volume_rank', 'trading_pair', 'volume_24h_usd', 'price_change_24h_pct', 'volatility_pct']
    top_pairs = df[display_cols].head(10)
    
    for _, row in top_pairs.iterrows():
        print(f"  {row['volume_rank']:2d}. {row['trading_pair']:12s} | "
              f"Vol: ${row['volume_24h_usd']:>10,.0f} | "
              f"Change: {row['price_change_24h_pct']:>+6.2f}% | "
              f"Vol: {row['volatility_pct']:>5.2f}%")

📊 Volume Analysis Complete:
  - Total pairs analyzed: 21
  - Top volume: $4,380,802,256
  - Date range: 15m candles

🏆 TOP 10 BY VOLUME:
----------------------------------------------------------------------
   1. ETH-USDT     | Vol: $4,380,802,256 | Change:  +0.80% | Vol:  0.26%
   2. BTC-USDT     | Vol: $3,864,052,880 | Change:  +0.36% | Vol:  0.19%
   3. SOL-USDT     | Vol: $1,624,360,790 | Change:  +1.17% | Vol:  0.44%
   4. WLD-USDT     | Vol: $1,260,812,640 | Change: +12.32% | Vol:  3.88%
   5. XRP-USDT     | Vol: $573,313,000 | Change:  +1.44% | Vol:  0.53%
   6. DOGE-USDT    | Vol: $489,970,725 | Change:  +1.88% | Vol:  0.44%
   7. ADA-USDT     | Vol: $225,605,574 | Change:  +1.99% | Vol:  0.61%
   8. LINK-USDT    | Vol: $185,966,285 | Change:  +1.46% | Vol:  0.38%
   9. AVAX-USDT    | Vol: $134,115,202 | Change:  +2.68% | Vol:  0.74%
  10. BNB-USDT     | Vol: $103,109,510 | Change:  +0.49% | Vol:  0.18%


# 📤 Generate and Send Email Volume Report
# Create professional HTML-formatted report and send via email notification system

In [5]:
# 🚀 Generate HTML Volume Report for Email Distribution
# Create comprehensive, professional HTML-formatted volume report

if not df.empty:
    # Select top pairs for the report
    top_pairs = df.head(TOP_PAIRS_COUNT)
    
    # Generate report timestamp
    report_time = datetime.now(timezone.utc).strftime('%Y-%m-%d %H:%M UTC')
    
    # Calculate summary statistics
    total_volume = df['volume_24h_usd'].sum()
    avg_price_change = df['price_change_24h_pct'].mean()
    max_gainer = df.loc[df['price_change_24h_pct'].idxmax()]
    max_loser = df.loc[df['price_change_24h_pct'].idxmin()]
    
    # Build compact HTML email report
    html_report = f"""<!DOCTYPE html>
<html>
<head>
    <meta charset="UTF-8">
    <title>Volume Screener Report</title>
    <style>
        body {{ font-family: 'Segoe UI', Arial, sans-serif; margin: 0; padding: 15px; background-color: #f8f9fa; line-height: 1.4; }}
        .container {{ max-width: 700px; margin: 0 auto; background-color: white; border-radius: 8px; box-shadow: 0 2px 4px rgba(0,0,0,0.1); overflow: hidden; }}
        .header {{ background: linear-gradient(135deg, #4a90e2 0%, #357abd 100%); color: white; padding: 20px; text-align: center; }}
        .header h1 {{ margin: 0; font-size: 24px; font-weight: 600; }}
        .header .subtitle {{ margin: 8px 0 0 0; opacity: 0.9; font-size: 14px; }}
        .content {{ padding: 20px; }}
        .summary {{ background-color: #f8f9fa; border-radius: 6px; padding: 15px; margin-bottom: 20px; border-left: 3px solid #4a90e2; }}
        .summary h3 {{ margin: 0 0 12px 0; color: #333; font-size: 18px; }}
        .summary-grid {{ display: grid; grid-template-columns: repeat(auto-fit, minmax(160px, 1fr)); gap: 12px; }}
        .summary-item {{ text-align: center; }}
        .summary-label {{ font-size: 11px; color: #666; text-transform: uppercase; letter-spacing: 0.5px; margin-bottom: 4px; }}
        .summary-value {{ font-size: 16px; font-weight: bold; color: #333; }}
        .positive {{ color: #28a745; }}
        .negative {{ color: #dc3545; }}
        .section-title {{ margin: 20px 0 12px 0; color: #333; font-size: 18px; font-weight: 600; }}
        table {{ width: 100%; border-collapse: collapse; font-size: 13px; }}
        th, td {{ padding: 8px 6px; text-align: left; border-bottom: 1px solid #dee2e6; }}
        th {{ background-color: #f8f9fa; font-weight: 600; color: #495057; font-size: 12px; }}
        .rank {{ font-weight: bold; color: #4a90e2; text-align: center; }}
        .pair {{ font-family: 'Courier New', monospace; font-weight: bold; }}
        .footer {{ background-color: #f8f9fa; padding: 15px; text-align: center; font-size: 11px; color: #666; line-height: 1.3; }}
        .change-positive {{ color: #28a745; font-weight: bold; }}
        .change-negative {{ color: #dc3545; font-weight: bold; }}
        .change-neutral {{ color: #6c757d; font-weight: bold; }}
        @media only screen and (max-width: 600px) {{
            .summary-grid {{ grid-template-columns: repeat(2, 1fr); }}
            th, td {{ padding: 6px 4px; font-size: 12px; }}
        }}
    </style>
</head>
<body>
    <div class="container">
        <div class="header">
            <h1>📊 Volume Screener Report</h1>
            <div class="subtitle">{report_time} • {CONNECTOR_NAME.replace('_', ' ').title()} • {INTERVAL} Timeframe</div>
        </div>
        
        <div class="content">
            <div class="summary">
                <h3>📈 Market Summary</h3>
                <div class="summary-grid">
                    <div class="summary-item">
                        <div class="summary-label">Total Volume</div>
                        <div class="summary-value">${total_volume:,.0f}</div>
                    </div>
                    <div class="summary-item">
                        <div class="summary-label">Avg Change</div>
                        <div class="summary-value {'positive' if avg_price_change > 0 else 'negative'}">{avg_price_change:+.2f}%</div>
                    </div>
                    <div class="summary-item">
                        <div class="summary-label">Top Gainer</div>
                        <div class="summary-value positive">{max_gainer['trading_pair']}</div>
                        <div style="font-size: 11px; margin-top: 2px; color: #28a745;">{max_gainer['price_change_24h_pct']:+.2f}%</div>
                    </div>
                    <div class="summary-item">
                        <div class="summary-label">Pairs Analyzed</div>
                        <div class="summary-value">{len(df)}</div>
                    </div>
                </div>
            </div>
            
            <h3 class="section-title">🏆 Top {TOP_PAIRS_COUNT} Trading Pairs by Volume</h3>
            <table>
                <thead>
                    <tr>
                        <th style="width: 8%;">#</th>
                        <th style="width: 20%;">Pair</th>
                        <th style="width: 25%;">24h Volume</th>
                        <th style="width: 15%;">Change</th>
                        <th style="width: 20%;">Price</th>
                        <th style="width: 12%;">Volatility</th>
                    </tr>
                </thead>
                <tbody>"""
    
    # Add top pairs to the HTML table with compact formatting
    for i, (_, row) in enumerate(top_pairs.iterrows(), 1):
        # Format volume in appropriate units
        volume_usd = row['volume_24h_usd']
        if volume_usd >= 1_000_000_000:
            volume_str = f"${volume_usd/1_000_000_000:.1f}B"
        elif volume_usd >= 1_000_000:
            volume_str = f"${volume_usd/1_000_000:.0f}M"
        elif volume_usd >= 1_000:
            volume_str = f"${volume_usd/1_000:.0f}K"
        else:
            volume_str = f"${volume_usd:.0f}"
        
        # Determine change class for styling
        change_pct = row['price_change_24h_pct']
        change_class = "change-positive" if change_pct > 0 else "change-negative" if change_pct < 0 else "change-neutral"
        
        # Format price appropriately
        price = row['price']
        if price >= 1000:
            price_str = f"${price:,.0f}"
        elif price >= 10:
            price_str = f"${price:.2f}"
        elif price >= 1:
            price_str = f"${price:.3f}"
        else:
            price_str = f"${price:.4f}"
        
        html_report += f"""
                    <tr>
                        <td class="rank">{i}</td>
                        <td class="pair">{row['trading_pair']}</td>
                        <td><strong>{volume_str}</strong></td>
                        <td class="{change_class}">{change_pct:+.2f}%</td>
                        <td>{price_str}</td>
                        <td>{row['volatility_pct']:.2f}%</td>
                    </tr>"""
    
    # Close the HTML report
    html_report += f"""
                </tbody>
            </table>
        </div>
        
        <div class="footer">
            Generated by QuantsLab Market Screener • {len(df)} trading pairs analyzed<br>
            Automated report from cached market data • {report_time}
        </div>
    </div>
</body>
</html>"""
    
    print("📤 Compact HTML email report generated successfully")
    print(f"📊 Report includes {len(top_pairs)} top pairs")
    print(f"💰 Total market volume: ${total_volume:,.0f}")
    
else:
    html_report = f"""<!DOCTYPE html>
<html>
<head>
    <meta charset="UTF-8">
    <title>Volume Screener Report - No Data</title>
    <style>
        body {{ font-family: 'Segoe UI', Arial, sans-serif; margin: 0; padding: 20px; background-color: #f8f9fa; }}
        .container {{ max-width: 500px; margin: 0 auto; background-color: white; border-radius: 8px; box-shadow: 0 2px 4px rgba(0,0,0,0.1); overflow: hidden; }}
        .header {{ background: linear-gradient(135deg, #dc3545 0%, #c82333 100%); color: white; padding: 20px; text-align: center; }}
        .content {{ padding: 20px; text-align: center; }}
        .footer {{ background-color: #f8f9fa; padding: 15px; text-align: center; font-size: 12px; color: #666; }}
    </style>
</head>
<body>
    <div class="container">
        <div class="header">
            <h1>⚠️ Volume Screener Report</h1>
            <div>{datetime.now(timezone.utc).strftime('%Y-%m-%d %H:%M UTC')}</div>
        </div>
        <div class="content">
            <h3>❌ No Market Data Available</h3>
            <p>No market data could be processed for analysis.</p>
            <p>Please check data collection tasks or fetch fresh data.</p>
        </div>
        <div class="footer">
            QuantsLab Market Screener
        </div>
    </div>
</body>
</html>"""
    
    print("⚠️  No data available for report generation")

# Display preview (first 500 characters)
print("\n📋 COMPACT EMAIL REPORT PREVIEW:")
print("=" * 50)
print(html_report[:500] + "..." if len(html_report) > 500 else html_report)
print("=" * 50)

📤 Compact HTML email report generated successfully
📊 Report includes 15 top pairs
💰 Total market volume: $13,370,789,540

📋 COMPACT EMAIL REPORT PREVIEW:
<!DOCTYPE html>
<html>
<head>
    <meta charset="UTF-8">
    <title>Volume Screener Report</title>
    <style>
        body { font-family: 'Segoe UI', Arial, sans-serif; margin: 0; padding: 15px; background-color: #f8f9fa; line-height: 1.4; }
        .container { max-width: 700px; margin: 0 auto; background-color: white; border-radius: 8px; box-shadow: 0 2px 4px rgba(0,0,0,0.1); overflow: hidden; }
        .header { background: linear-gradient(135deg, #4a90e2 0%, #357abd 100%); color: white; pad...


In [ ]:
# 📧 Send Volume Report via All Enabled Notifiers
# Distribute the report through all notification systems with optional recipient override

# Create notification message
notification = NotificationMessage(
    title="📊 Daily Volume Screener Report",
    message=html_report,
    level="info"
)

print("📤 Sending volume report via all enabled notification systems...")

# Display override information
override_info = []
if CUSTOM_EMAIL_RECIPIENTS:
    override_info.append(f"Email: {', '.join(CUSTOM_EMAIL_RECIPIENTS)}")
if CUSTOM_TELEGRAM_CHAT_IDS:
    override_info.append(f"Telegram: {len(CUSTOM_TELEGRAM_CHAT_IDS)} chat IDs")
if CUSTOM_SLACK_CHANNELS:
    override_info.append(f"Slack: {', '.join(CUSTOM_SLACK_CHANNELS)}")
if CUSTOM_DISCORD_WEBHOOKS:
    override_info.append(f"Discord: {len(CUSTOM_DISCORD_WEBHOOKS)} webhooks")

if override_info:
    print(f"   📧 Custom recipients: {'; '.join(override_info)}")
else:
    print(f"   📧 Using default .env configured recipients for all services")

# Send notifications with recipient overrides
try:
    results = {}
    
    # ============================================
    # EMAIL NOTIFIER WITH RECIPIENT OVERRIDE
    # ============================================
    if 'email' in enabled_notifiers:
        email_notifier = notification_manager.get_notifier('email')
        if email_notifier:
            email_success = await email_notifier.send_notification(
                message=notification,
                to_addresses=CUSTOM_EMAIL_RECIPIENTS  # None = use defaults
            )
            results['email'] = email_success
    
    # ============================================
    # TELEGRAM NOTIFIER WITH CHAT ID OVERRIDE
    # ============================================
    if 'telegram' in enabled_notifiers:
        telegram_notifier = notification_manager.get_notifier('telegram')
        if telegram_notifier:
            telegram_success = await telegram_notifier.send_notification(
                message=notification,
                chat_ids=CUSTOM_TELEGRAM_CHAT_IDS  # None = use default
            )
            results['telegram'] = telegram_success
    
    # ============================================
    # SLACK NOTIFIER WITH CHANNEL/WEBHOOK OVERRIDE
    # ============================================
    if 'slack' in enabled_notifiers:
        slack_notifier = notification_manager.get_notifier('slack')
        if slack_notifier:
            slack_success = await slack_notifier.send_notification(
                message=notification,
                channels=CUSTOM_SLACK_CHANNELS,      # None = use default
                webhook_urls=CUSTOM_SLACK_WEBHOOKS   # None = use default
            )
            results['slack'] = slack_success
    
    # ============================================
    # DISCORD NOTIFIER WITH WEBHOOK OVERRIDE
    # ============================================
    if 'discord' in enabled_notifiers:
        discord_notifier = notification_manager.get_notifier('discord')
        if discord_notifier:
            discord_success = await discord_notifier.send_notification(
                message=notification,
                webhook_urls=CUSTOM_DISCORD_WEBHOOKS  # None = use default
            )
            results['discord'] = discord_success
    
    # ============================================
    # RESULTS SUMMARY
    # ============================================
    print(f"\n📊 Notification Delivery Results:")
    print("-" * 50)
    
    for service, success in results.items():
        status_emoji = "✅" if success else "❌"
        status_text = "Delivered" if success else "Failed"
        service_name = service.capitalize()
        print(f"  {status_emoji} {service_name}: {status_text}")
    
    # Overall summary
    successful_deliveries = sum(results.values())
    total_services = len(results)
    print(f"\n📈 Delivery Summary: {successful_deliveries}/{total_services} services successful")
    
    if successful_deliveries == 0:
        print("⚠️ No notifications were delivered successfully")
        print("💡 Check your notification configurations in .env file")
    elif successful_deliveries == total_services:
        print("🎉 Volume report successfully sent via all enabled services!")
        print("📧 Check your inboxes/channels for the HTML-formatted reports")
    else:
        print("⚠️ Some notifications failed - check configurations for failed services")
        
except Exception as e:
    print(f"❌ Error sending notifications: {e}")
    print("💡 Check your notification configurations and network connection")

# 📊 Generate and Email Volume Chart
# Create professional volume visualization chart and send as email attachment

In [ ]:
# 🎨 Generate and Send Volume Chart as Email Attachment
# Create professional volume visualization chart optimized for email

if not df.empty and len(df) >= 10:
    print("🎨 Creating volume chart for email attachment...")
    
    # Select top 10 pairs by volume for visualization
    top_10 = df.nlargest(10, 'volume_24h_usd')
    
    # Create horizontal bar chart
    fig = go.Figure()
    
    # Generate clean pair labels
    pair_labels = [row['trading_pair'] for _, row in top_10.iterrows()]
    
    # Create professional color scheme for email
    colors = ['#E74C3C' if change < 0 else '#27AE60' if change > 0 else '#95A5A6' 
              for change in top_10['price_change_24h_pct']]
    
    fig.add_trace(go.Bar(
        y=pair_labels,
        x=top_10['volume_24h_usd'],
        orientation='h',
        marker=dict(
            color=colors,
            opacity=0.8,
            line=dict(color='rgba(0,0,0,0.1)', width=1)
        ),
        text=[f"${vol/1e6:.1f}M" for vol in top_10['volume_24h_usd']],
        textposition='inside',
        textfont=dict(color='white', size=12, family='Arial Bold'),
        hovertemplate=(
            '<b>%{customdata[0]}</b><br>'
            'Volume: $%{x:,.0f}<br>'
            'Change: %{customdata[1]:+.2f}%<br>'
            'Price: $%{customdata[2]:.4f}'
            '<extra></extra>'
        ),
        customdata=list(zip(
            top_10['trading_pair'],
            top_10['price_change_24h_pct'],
            top_10['price']
        ))
    ))
    
    # Customize layout for professional email presentation
    fig.update_layout(
        title=dict(
            text=f"Top 10 Trading Pairs by 24h Volume<br><sub>QuantsLab Market Screener - {CONNECTOR_NAME.replace('_', ' ').title()} ({datetime.now(timezone.utc).strftime('%Y-%m-%d %H:%M UTC')})</sub>",
            x=0.5,
            xanchor='center',
            font=dict(size=20, family='Arial Black', color='#2C3E50')
        ),
        xaxis_title="24-Hour Trading Volume (USD)",
        yaxis_title="Trading Pairs",
        template="plotly_white",
        height=700,
        width=1200,
        margin=dict(l=140, r=100, t=120, b=80),
        xaxis=dict(
            tickformat='$,.0s',
            showgrid=True,
            gridwidth=1,
            gridcolor='#E8E8E8',
            tickfont=dict(size=11),
            title_font=dict(size=14, family='Arial Bold')
        ),
        yaxis=dict(
            showgrid=False,
            tickfont=dict(size=12, family='Arial Bold'),
            title_font=dict(size=14, family='Arial Bold')
        ),
        plot_bgcolor='white',
        paper_bgcolor='#FAFAFA',
        font=dict(family='Arial', size=12, color='#2C3E50')
    )
    
    # Add professional footer annotation
    fig.add_annotation(
        text=f"📊 Generated by QuantsLab • Data from {len(df)} trading pairs • {INTERVAL} timeframe",
        xref="paper", yref="paper",
        x=0.5, y=-0.12, xanchor="center", yanchor="top",
        showarrow=False,
        font=dict(size=11, color="#7F8C8D", family="Arial")
    )
    
    # Display chart in notebook
    fig.show()
    
    # ============================================
    # SAVE AND SEND CHART VIA EMAIL ATTACHMENT
    # ============================================
    chart_png_path = '/tmp/volume_chart_email.png'
    
    print("💾 Generating high-quality PNG chart for email attachment...")
    
    try:
        # Save as high-quality PNG for email
        fig.write_image(
            chart_png_path, 
            width=1200, 
            height=700, 
            scale=2,  # High DPI for crisp email display
            engine='kaleido'
        )
        print(f"✅ High-quality PNG chart saved: {chart_png_path}")
        
        # Send via Email
        email_notifier = notification_manager.get_notifier('email')
        
        if email_notifier:
            print("📧 Sending chart via email as attachment...")
            if CUSTOM_EMAIL_RECIPIENTS:
                print(f"   📧 Custom recipients: {', '.join(CUSTOM_EMAIL_RECIPIENTS)}")
            
            # Create professional email with chart attachment
            chart_email_html = f"""<!DOCTYPE html>
<html>
<body style="font-family: Arial, sans-serif; margin: 0; padding: 20px; background-color: #f5f5f5;">
    <div style="max-width: 600px; margin: 0 auto; background-color: white; border-radius: 10px; padding: 30px; box-shadow: 0 4px 6px rgba(0,0,0,0.1);">
        <h2 style="color: #2C3E50; text-align: center; margin-bottom: 20px;">📊 Volume Analysis Chart</h2>
        <p style="color: #34495E; font-size: 16px; line-height: 1.6;">Dear Team,</p>
        <p style="color: #34495E; font-size: 16px; line-height: 1.6;">
            Please find attached the latest volume analysis chart showing the top 10 trading pairs by 24-hour volume.
        </p>
        <div style="background-color: #EBF3FD; border-radius: 8px; padding: 20px; margin: 20px 0; border-left: 4px solid #3498DB;">
            <h3 style="margin-top: 0; color: #2C3E50;">📈 Chart Highlights:</h3>
            <ul style="color: #34495E; margin: 10px 0;">
                <li><strong>Top Performer:</strong> {top_10.iloc[0]['trading_pair']} (${top_10.iloc[0]['volume_24h_usd']/1e6:.1f}M volume)</li>
                <li><strong>Total Volume:</strong> ${top_10['volume_24h_usd'].sum():,.0f}</li>
                <li><strong>Exchange:</strong> {CONNECTOR_NAME.replace('_', ' ').title()}</li>
                <li><strong>Timeframe:</strong> {INTERVAL} candles</li>
                <li><strong>Colors:</strong> 🟢 Green = Price Up, 🔴 Red = Price Down</li>
            </ul>
        </div>
        <p style="color: #34495E; font-size: 16px; line-height: 1.6;">
            The attached chart provides a visual representation of market leadership and can be used for trading decisions and market analysis.
        </p>
        <p style="color: #7F8C8D; font-size: 12px; text-align: center; margin-top: 30px; border-top: 1px solid #E8E8E8; padding-top: 20px;">
            📊 Generated by QuantsLab Market Screener<br>
            Report Date: {datetime.now(timezone.utc).strftime('%Y-%m-%d %H:%M UTC')}
        </p>
    </div>
</body>
</html>"""
            
            # Create notification message for chart attachment
            chart_notification = NotificationMessage(
                title=f"📊 Volume Chart - {CONNECTOR_NAME.replace('_', ' ').title()} Market Analysis",
                message=chart_email_html,
                level="info"
            )
            
            # Send email with chart attachment using recipient override
            chart_success = await email_notifier.send_attachment(
                message=chart_notification,
                attachment_path=chart_png_path,
                attachment_name="volume_analysis_chart.png",
                to_addresses=CUSTOM_EMAIL_RECIPIENTS  # None = use defaults
            )
            
            print(f"  📊 Chart Email: {'✅ Sent successfully!' if chart_success else '❌ Failed to send'}")
            
            if chart_success:
                print("🎉 Volume chart delivered via email with professional formatting!")
                print("📧 Check your email for the chart attachment and analysis")
            else:
                print("⚠️ Failed to send chart via email - check SMTP configuration")
                print(f"💾 Chart saved locally at: {chart_png_path}")
                
        else:
            print("⚠️ Email notifier not configured")
            print(f"💾 PNG chart saved locally: {chart_png_path}")
            print("💡 Configure email settings in .env to send chart attachments")
            
    except ImportError as e:
        print(f"❌ Kaleido not installed properly: {e}")
        print("💡 Try: pip install --upgrade kaleido")
    except Exception as e:
        print(f"❌ Error generating or sending chart: {e}")
        print("💡 Check kaleido installation and email configuration")
        
else:
    print("⚠️ Insufficient data to generate volume chart")
    if df.empty:
        print("💡 No data available - ensure data collection tasks are running")
    else:
        print(f"💡 Only {len(df)} pairs available - need at least 10 for chart generation")

# 📋 Generate and Email CSV Reports
# Create detailed data export and send as email attachment

In [ ]:
# 📊 Generate and Send Comprehensive CSV Report via Email
# Create detailed data export for advanced analysis and email as attachment

if not df.empty:
    print("📋 Generating detailed CSV report for email attachment...")
    
    # Prepare comprehensive dataset
    detailed_df = df.copy()
    
    # Add timestamp for the report
    detailed_df['report_timestamp'] = datetime.now(timezone.utc).strftime('%Y-%m-%d %H:%M:%S UTC')
    
    # Calculate additional derived metrics
    detailed_df['volume_24h_usd_millions'] = detailed_df['volume_24h_usd'] / 1_000_000
    detailed_df['market_cap_rank'] = detailed_df['volume_rank']  # Use volume rank as proxy
    
    # Add performance categories
    detailed_df['performance_category'] = detailed_df['price_change_24h_pct'].apply(
        lambda x: 'Strong Gain' if x > 5 else 
                  'Moderate Gain' if x > 1 else 
                  'Stable' if x > -1 else 
                  'Moderate Loss' if x > -5 else 'Strong Loss'
    )
    
    # Add volume categories
    volume_quantiles = detailed_df['volume_24h_usd'].quantile([0.25, 0.5, 0.75])
    detailed_df['volume_category'] = detailed_df['volume_24h_usd'].apply(
        lambda x: 'Very High' if x > volume_quantiles[0.75] else
                  'High' if x > volume_quantiles[0.5] else
                  'Medium' if x > volume_quantiles[0.25] else 'Low'
    )
    
    # Reorder columns for better presentation
    column_order = [
        'volume_rank', 'trading_pair', 'connector_name', 'price', 
        'volume_24h_usd', 'volume_24h_usd_millions', 'volume_category',
        'price_change_24h_pct', 'performance_category', 'volatility_pct',
        'high_24h', 'low_24h', 'position_in_range', 'volume_trend_pct',
        'composite_score', 'data_points', 'report_timestamp'
    ]
    
    # Add any remaining columns
    remaining_cols = [col for col in detailed_df.columns if col not in column_order]
    final_columns = column_order + remaining_cols
    detailed_df = detailed_df[final_columns]
    
    # Create timestamped filename
    timestamp_str = datetime.now(timezone.utc).strftime("%Y%m%d_%H%M")
    csv_path = f'/tmp/volume_screener_report_{timestamp_str}.csv'
    
    # Save to CSV with proper formatting
    detailed_df.to_csv(csv_path, index=False, float_format='%.8f')
    
    print(f"✅ CSV report generated: {len(detailed_df)} records")
    print(f"💾 File location: {csv_path}")
    
    # Send via Email if available
    email_notifier = notification_manager.get_notifier('email')
    
    if email_notifier:
        print("📧 Sending CSV report via email as attachment...")
        if CUSTOM_EMAIL_RECIPIENTS:
            print(f"   📧 Custom recipients: {', '.join(CUSTOM_EMAIL_RECIPIENTS)}")
        
        # Create professional email for CSV attachment
        csv_email_html = f"""<!DOCTYPE html>
<html>
<body style="font-family: Arial, sans-serif; margin: 0; padding: 20px; background-color: #f5f5f5;">
    <div style="max-width: 600px; margin: 0 auto; background-color: white; border-radius: 10px; padding: 30px; box-shadow: 0 4px 6px rgba(0,0,0,0.1);">
        <h2 style="color: #2C3E50; text-align: center; margin-bottom: 20px;">📋 Volume Screener Data Export</h2>
        <p style="color: #34495E; font-size: 16px; line-height: 1.6;">Dear Team,</p>
        <p style="color: #34495E; font-size: 16px; line-height: 1.6;">
            Please find attached the complete volume screener dataset in CSV format for detailed analysis.
        </p>
        <div style="background-color: #E8F6F3; border-radius: 8px; padding: 20px; margin: 20px 0; border-left: 4px solid #27AE60;">
            <h3 style="margin-top: 0; color: #2C3E50;">📊 Dataset Summary:</h3>
            <ul style="color: #34495E; margin: 10px 0; line-height: 1.8;">
                <li><strong>Total Records:</strong> {len(detailed_df)} trading pairs</li>
                <li><strong>Top Volume:</strong> ${detailed_df['volume_24h_usd'].max():,.0f}</li>
                <li><strong>Exchange:</strong> {CONNECTOR_NAME.replace('_', ' ').title()}</li>
                <li><strong>Timeframe:</strong> {INTERVAL} candles</li>
                <li><strong>Categories:</strong> Performance & Volume rankings included</li>
                <li><strong>Metrics:</strong> Volume, Price Change, Volatility, Trends</li>
            </ul>
        </div>
        <div style="background-color: #FEF9E7; border-radius: 8px; padding: 15px; margin: 20px 0; border-left: 4px solid #F39C12;">
            <h4 style="margin-top: 0; color: #2C3E50;">💡 Data Usage Tips:</h4>
            <p style="color: #34495E; margin: 5px 0; font-size: 14px;">• Open in Excel or Google Sheets for pivot tables and analysis</p>
            <p style="color: #34495E; margin: 5px 0; font-size: 14px;">• Use volume_category and performance_category for filtering</p>
            <p style="color: #34495E; margin: 5px 0; font-size: 14px;">• Composite_score column provides overall ranking metric</p>
        </div>
        <p style="color: #34495E; font-size: 16px; line-height: 1.6;">
            This comprehensive dataset includes all calculated metrics and can be used for advanced market analysis, 
            backtesting, and strategy development.
        </p>
        <p style="color: #7F8C8D; font-size: 12px; text-align: center; margin-top: 30px; border-top: 1px solid #E8E8E8; padding-top: 20px;">
            📊 Generated by QuantsLab Market Screener<br>
            Report Date: {datetime.now(timezone.utc).strftime('%Y-%m-%d %H:%M UTC')}<br>
            Complete dataset with {len(detailed_df)} trading pairs
        </p>
    </div>
</body>
</html>"""
        
        try:
            # Create notification message for CSV attachment
            csv_notification = NotificationMessage(
                title=f"📋 Volume Screener Data Export - {datetime.now(timezone.utc).strftime('%Y-%m-%d')}",
                message=csv_email_html,
                level="info"
            )
            
            # Send email with CSV attachment using recipient override
            doc_success = await email_notifier.send_attachment(
                message=csv_notification,
                attachment_path=csv_path,
                attachment_name=f"volume_screener_{timestamp_str}.csv",
                to_addresses=CUSTOM_EMAIL_RECIPIENTS  # None = use defaults
            )
            
            print(f"📋 CSV Email: {'✅ Sent successfully!' if doc_success else '❌ Failed to send'}")
            
            if doc_success:
                print("🎉 Detailed CSV report successfully delivered via email!")
                print("📧 Check your email for the comprehensive dataset attachment")
            else:
                print("⚠️ CSV delivery failed - check email SMTP configuration")
        except Exception as e:
            print(f"❌ Error sending CSV via email: {e}")
            print("💾 CSV file saved locally and ready for manual distribution")
            
    else:
        print("⚠️ Email notifier not configured for CSV sending")
        print(f"💾 CSV file saved locally and ready for manual distribution")
        print("💡 Configure email settings in .env to send CSV attachments")
    
    # Display sample of the data
    print(f"\n📋 CSV Report Preview (Top 5 rows):")
    print("-" * 100)
    preview_cols = ['volume_rank', 'trading_pair', 'volume_24h_usd', 'price_change_24h_pct', 'performance_category']
    available_cols = [col for col in preview_cols if col in detailed_df.columns]
    print(detailed_df[available_cols].head().to_string(index=False))
    print("-" * 100)
    
else:
    print("⚠️ No data available for CSV report generation")
    print("💡 Ensure data collection tasks are running and populating the cache")

# 🧹 Cleanup and Finalization
# Clean up resources and temporary files

In [ ]:
# 🔒 Cleanup and Summary
# Clean up temporary files and provide execution summary

print("✅ Email volume screener analysis completed successfully")

# Optional: Clean up temporary files
import os
temp_files = ['/tmp/volume_chart_email.png']
csv_files = [f for f in os.listdir('/tmp') if f.startswith('volume_screener_report_') and f.endswith('.csv')]

cleanup_count = 0
for file_path in temp_files:
    if os.path.exists(file_path):
        try:
            os.remove(file_path)
            cleanup_count += 1
            print(f"🗑️  Cleaned up: {file_path}")
        except Exception as e:
            print(f"⚠️  Could not remove {file_path}: {e}")

# Optionally clean up old CSV files (keep only the most recent)
if len(csv_files) > 1:
    csv_files_full = [f'/tmp/{f}' for f in csv_files]
    csv_files_full.sort(key=lambda x: os.path.getctime(x))
    
    # Remove all but the most recent
    for old_csv in csv_files_full[:-1]:
        try:
            os.remove(old_csv)
            cleanup_count += 1
            print(f"🗑️  Cleaned up old CSV: {os.path.basename(old_csv)}")
        except Exception as e:
            print(f"⚠️  Could not remove {old_csv}: {e}")

if cleanup_count > 0:
    print(f"🧹 Cleaned up {cleanup_count} temporary files")
else:
    print("🧹 No temporary files to clean up")

# Execution Summary
print("\n📊 EXECUTION SUMMARY")
print("=" * 50)
print(f"📈 Data Source: CLOB Cached Data ({CONNECTOR_NAME})")
print(f"⏰ Timeframe: {INTERVAL} candles")
print(f"💹 Pairs Analyzed: {len(df) if not df.empty else 0}")
print(f"📧 Email Report Sent: {'✅ Yes' if 'email' in enabled_notifiers else '❌ Email not configured'}")
print(f"📊 Charts Generated: {'✅ Yes' if not df.empty and len(df) >= 10 else '❌ Insufficient data'}")
print(f"📋 CSV Export: {'✅ Yes' if not df.empty else '❌ No data available'}")

print("\n🎉 Email volume screener report generation complete!")
if 'email' in enabled_notifiers:
    print("📧 Check your email for delivered reports, charts, and data files")
    print("💡 Reports are delivered as professional HTML emails with attachments")
else:
    print("💡 Configure email SMTP settings in .env to receive automated reports")

print("\n🔧 Next Steps:")
print("  • Run data collection tasks regularly for fresh data")
print("  • Adjust TOP_PAIRS_COUNT for different report sizes")
print("  • Set FETCH_FRESH_DATA = True for real-time analysis")
print("  • Schedule this notebook as a task for automated email reports")
print("  • Customize email templates and styling as needed")

# 🛠️ Setup and Configuration Guide

## 📧 Recipient Override Examples

### 1. Basic Usage (Use .env defaults)
```python
# All None = use configured .env recipients
CUSTOM_EMAIL_RECIPIENTS = None
CUSTOM_TELEGRAM_CHAT_IDS = None  
CUSTOM_SLACK_CHANNELS = None
CUSTOM_DISCORD_WEBHOOKS = None
```

### 2. Email Override Examples
```python
# Send to specific email addresses
CUSTOM_EMAIL_RECIPIENTS = ["analyst@company.com", "trader@company.com"]

# Send to distribution lists
CUSTOM_EMAIL_RECIPIENTS = ["trading-team@company.com", "risk-management@company.com"]
```

### 3. Telegram Override Examples
```python
# Send to multiple chats (private chats, groups, channels)
CUSTOM_TELEGRAM_CHAT_IDS = [
    "123456789",           # Private chat with user
    "-1001234567890",      # Group chat (starts with -100)
    "@trading_alerts"      # Public channel username
]
```

### 4. Slack Override Examples
```python
# Send to specific channels
CUSTOM_SLACK_CHANNELS = ["#trading-alerts", "#risk-management", "#general"]

# Send to multiple workspaces
CUSTOM_SLACK_WEBHOOKS = [
    "https://hooks.slack.com/services/T00/B00/XXX",  # Workspace 1
    "https://hooks.slack.com/services/T11/B11/YYY"   # Workspace 2
]

# Combine both (sends to each channel in each workspace)
CUSTOM_SLACK_CHANNELS = ["#alerts", "#trading"]
CUSTOM_SLACK_WEBHOOKS = ["https://hooks.slack.com/services/T00/B00/XXX"]
```

### 5. Discord Override Examples
```python
# Send to multiple Discord servers/channels
CUSTOM_DISCORD_WEBHOOKS = [
    "https://discord.com/api/webhooks/123/ABC-trading-channel",
    "https://discord.com/api/webhooks/456/DEF-alerts-channel",
    "https://discord.com/api/webhooks/789/GHI-general-channel"
]
```

### 6. Mixed Override Scenarios
```python
# Executive Summary: Email only to leadership
CUSTOM_EMAIL_RECIPIENTS = ["ceo@company.com", "cto@company.com"]
CUSTOM_TELEGRAM_CHAT_IDS = None  # Skip Telegram
CUSTOM_SLACK_CHANNELS = None     # Skip Slack

# Trading Team: All channels with custom recipients
CUSTOM_EMAIL_RECIPIENTS = ["traders@company.com"]
CUSTOM_TELEGRAM_CHAT_IDS = ["-1001234567890"]  # Trading group
CUSTOM_SLACK_CHANNELS = ["#trading-floor"]
CUSTOM_DISCORD_WEBHOOKS = ["https://discord.com/api/webhooks/123/trading"]

# Emergency Alerts: Broadcast everywhere
CUSTOM_EMAIL_RECIPIENTS = ["alerts@company.com", "oncall@company.com"]
CUSTOM_TELEGRAM_CHAT_IDS = ["123", "456", "-1001234567890"]
CUSTOM_SLACK_CHANNELS = ["#alerts", "#general", "#trading"]  
CUSTOM_DISCORD_WEBHOOKS = [
    "https://discord.com/api/webhooks/111/alerts",
    "https://discord.com/api/webhooks/222/general"
]
```

## 📱 Platform-Specific Features

### **Email**
- **Multiple Recipients**: Send to unlimited email addresses
- **Professional Formatting**: HTML with CSS styling
- **Attachments**: Charts and CSV files included
- **Corporate Integration**: Works with any SMTP server

### **Telegram**
- **Multiple Chats**: Private, group, and channel support
- **Rich Formatting**: HTML/Markdown support
- **Media Support**: Send charts via `send_photo()` and CSV via `send_document()`
- **Chat Types**: Users (numbers), Groups (-100...), Channels (@username)

### **Slack**
- **Multi-Workspace**: Send to different Slack workspaces
- **Channel Targeting**: Override default channel
- **Rich Formatting**: Slack markup and emojis
- **Corporate Integration**: Works with enterprise Slack

### **Discord**
- **Multi-Server**: Send to multiple Discord servers
- **Rich Embeds**: Color-coded messages with formatting
- **Webhook Flexibility**: Easy setup, no bot required
- **Community Integration**: Perfect for trading communities

## ⚙️ Configuration Files

### .env Configuration (Default Recipients)
```bash
# Email Configuration
EMAIL_ENABLED=true
SMTP_TO_EMAIL=default@company.com

# Telegram Configuration  
TELEGRAM_ENABLED=true
TELEGRAM_CHAT_ID=123456789

# Slack Configuration
SLACK_ENABLED=true
SLACK_WEBHOOK_URL=https://hooks.slack.com/services/T00/B00/XXX
SLACK_CHANNEL=#general

# Discord Configuration
DISCORD_ENABLED=true
DISCORD_WEBHOOK_URL=https://discord.com/api/webhooks/123/ABC
```

### Override Usage Priority
1. **Custom Variables** (highest priority): `CUSTOM_*_RECIPIENTS`
2. **Method Parameters**: Direct notifier calls
3. **Environment Variables** (lowest priority): `.env` defaults

## 🔧 Advanced Usage Patterns

### Pattern 1: Conditional Distribution
```python
# Send to different recipients based on market conditions
if avg_price_change > 5:  # High volatility
    CUSTOM_EMAIL_RECIPIENTS = ["executives@company.com"]
    CUSTOM_TELEGRAM_CHAT_IDS = ["-1001234567890"]  # Urgent alerts group
elif avg_price_change < -3:  # Market decline
    CUSTOM_SLACK_CHANNELS = ["#risk-management", "#trading"]
else:  # Normal conditions
    # Use defaults - all None
    pass
```

### Pattern 2: Time-Based Distribution  
```python
import datetime
hour = datetime.now().hour

if 9 <= hour <= 17:  # Business hours
    CUSTOM_EMAIL_RECIPIENTS = ["traders@company.com"]
    CUSTOM_SLACK_CHANNELS = ["#trading-floor"]
else:  # After hours
    CUSTOM_TELEGRAM_CHAT_IDS = ["-1001234567890"]  # Night shift group
    CUSTOM_DISCORD_WEBHOOKS = ["https://discord.com/api/webhooks/123/night-alerts"]
```

### Pattern 3: Department-Specific Reports
```python
# Different reports for different teams
def send_to_trading_team():
    return {
        'email': ["traders@company.com"],
        'telegram': ["-1001234567890"],
        'slack': ["#trading-floor"]
    }

def send_to_executives():
    return {
        'email': ["ceo@company.com", "cfo@company.com"],
        'slack': ["#executive-updates"]
    }

# Usage
if report_type == "detailed":
    recipients = send_to_trading_team()
elif report_type == "summary": 
    recipients = send_to_executives()
```

## 🚀 Best Practices

1. **Test Gradually**: Start with one override, then expand
2. **Document Recipients**: Keep a list of who receives what
3. **Respect Limits**: Each platform has rate limits and recipient limits
4. **Error Handling**: Always check delivery results
5. **Privacy**: Be mindful of sensitive data in group chats
6. **Compliance**: Ensure email distribution follows company policies
7. **Backup Channels**: Have multiple notification paths for critical alerts